# Variational Autoencoder

In [ ]:
using Flux, Flux.Data.MNIST
using Flux: throttle, params
using Juno: @progress


In [ ]:
# Load data, binarise it, and partition into mini-batches of M.
X = float.(hcat(vec.(MNIST.images())...)) .> 0.5
N, M = size(X, 2), 100
data = [X[:,i] for i in Iterators.partition(1:N,M)]

Po tym jak wyjaśniliśmy sobie czym jest zwykły autoencoder możemy przejść do opisania jego bardziej zaawansowanej (i ciekawszej) formy - Variational Autoencodera (VAE).

Klasyczny autoencoder pozwala nam na zakodowanie danych (najczęściej obrazu) w formie wektora zmiennych ukrytych <i>(latent variables)</i>:

[![](http://kvfrans.com/content/images/2016/08/autoenc.jpg)](http://kvfrans.com/variational-autoencoders-explained/)

Taki model pozwala na całkiem efektywną kompresję wejściowych danych. Jednak co należy zrobić gdy chcielibyśmy generować noweobrazy na podstawie danych wejściowych a nie jedynie je odtwarzać?


Variational Autoencoder jest klasą modeli, która na to pozwala. Intuicja  za nimi stojąca jest prosta, sieć dekodująca zamiast odtwarzać obraz w skali 1:1 dodaje do nich pewną losowość, dzięki której wyjściowy obraz jest nieznacznie zmieniony w stosunku do obrazu wejściowego.

Otrzymanie tej losowości jest możliwe dzięki wygenerowaniu przez sieć kodującą nie tyle wektora zmiennych ukrytych, co wektora średnich i odchyleń standardowych zmiennych ukrytych z których następnie losowane są zmienne ukryte wykorzystane przez sieć dekodującą:

[![](http://kvfrans.com/content/images/2016/08/vae.jpg)](http://kvfrans.com/variational-autoencoders-explained/)

Zdefiniujmy taki model:

In [ ]:
################################# Define Model #################################

# Latent dimensionality, # hidden units.
Dz, Dh = 5, 500

# Components of recognition model / "encoder" MLP.
A, μ, logσ = Dense(28^2, Dh, tanh), Dense(Dh, Dz), Dense(Dh, Dz)
g(X) = (h = A(X); (μ(h), logσ(h)))
z(μ, logσ) = μ + exp(logσ) * randn()

In [ ]:
# Generative model / "decoder" MLP.
f = Chain(Dense(Dz, Dh, tanh), Dense(Dh, 28^2, σ))

Przejdźmy teraz do zdefiniowania funkcji straty i procesu uczenia się modelu. Zacznijmy od prostej, technicznej definicji:

In [ ]:
####################### Define ways of doing things with the model. #######################
# Extend distributions slightly to have a numerically stable logpdf for `p` close to 1 or 0.
using Distributions
import Distributions: logpdf
logpdf(b::Bernoulli, y::Bool) = y * log(b.p + eps()) + (1 - y) * log(1 - b.p + eps())

i zastanówmy jak powinna wyglądać funkcja straty tego modelu:

In [ ]:
# KL-divergence between approximation posterior and N(0, 1) prior.
kl_q_p(μ, logσ) = 0.5 * sum(exp.(2 .* logσ) + μ.^2 - 1 .+ logσ.^2)

# logp(x|z) - conditional probability of data given latents.
logp_x_z(x, z) = sum(logpdf.(Bernoulli.(f(z)), x))

# Monte Carlo estimator of mean ELBO using M samples.
L̄(X) = ((μ̂, logσ̂) = g(X); (logp_x_z(X, z.(μ̂, logσ̂)) - kl_q_p(μ̂, logσ̂)) / M)

loss(X) = -L̄(X) + 0.01 * sum(x->sum(x.^2), params(f))


Wiedząc jak powinien wyglądać model możemy zacząć go uczyć:

In [ ]:

################################# Learn Parameters ##############################

evalcb = throttle(() -> @show(-L̄(X[:, rand(1:N, M)])), 30)
opt = ADAM(params(A, μ, logσ, f))
@progress for i = 1:20
  info("Epoch $i")
  Flux.train!(loss, zip(data), opt, cb=evalcb)
end


I na koniec sprawdzić czy sieć poprawnie generuje obrazy:

In [ ]:
################################# Sample Output ##############################

using Images

# Sample from the learned model.
modelsample() = rand.(Bernoulli.(f(z.(zeros(Dz), zeros(Dz)))))


img(x) = Gray.(reshape(x, 28, 28))

sample = hcat(img.([modelsample() for i = 1:10])...)
save("sample.png", sample)